In [1]:
# Read in a WAV and find the freq's
import pyaudio
import wave
import numpy as np
import json

chunk = 2048
l =[]

In [2]:
# open up a wave
wf = wave.open('3A_1.wav', 'rb')
swidth = wf.getsampwidth()
RATE = wf.getframerate()

print('チャンネル数:', wf.getnchannels()) # モノラルなら1，ステレオなら2
print('サンプル幅:', wf.getsampwidth()) # バイト数 (1byte=8bit)
print('サンプリング周波数:', wf.getframerate()) # CDは44100Hz
print('フレーム数:', wf.getnframes()) # サンプリング周波数で割れば時間
print('パラメータ:', wf.getparams()) # 上記+αのパラメータをタプルで返す

# use a Blackman window
window = np.blackman(chunk)
# open stream
p = pyaudio.PyAudio()
stream = p.open(format =
                p.get_format_from_width(wf.getsampwidth()),
                channels = wf.getnchannels(),
                rate = RATE,
                output = True)


""" 
   format  : ストリームを読み書きするときのデータ型
   channels: ステレオかモノラルかの選択 1でモノラル 2でステレオ
   rate    : サンプル周波数
   output  : 出力モード

"""

チャンネル数: 1
サンプル幅: 2
サンプリング周波数: 44100
フレーム数: 294912
パラメータ: _wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=294912, comptype='NONE', compname='not compressed')


' \n   format  : ストリームを読み書きするときのデータ型\n   channels: ステレオかモノラルかの選択 1でモノラル 2でステレオ\n   rate    : サンプル周波数\n   output  : 出力モード\n\n'

In [3]:
# read some data
data = wf.readframes(chunk)

In [4]:
# play stream and find the frequency of each chunk
# dataの要素数＝＝chunk*swidth(2048*サンプル幅)になるまで
while len(data) == chunk*swidth:
    # write data out to the audio stream
    stream.write(data)
    
    # unpack the data and times by the hamming window
    indata = np.array(wave.struct.unpack("%dh"%(len(data)/swidth),\
                                         data))*window
    
    # Take the fft and square each value
    fftData=abs(np.fft.rfft(indata))**2
    
    # find the maximum
    which = fftData[1:].argmax() + 1
    
    # use quadratic interpolation around the max
    if which != len(fftData)-1:
        y0,y1,y2 = np.log(fftData[which-1:which+2:])
        x1 = (y2 - y0) * .5 / (2 * y1 - y2 - y0)
        # find the frequency and output it
        thefreq = (which+x1)*RATE/chunk
        hz = "The freq is {} Hz." .format (thefreq)
        print(hz)
        if np.isnan(thefreq):
            print("nan")
        else:
            l.append(thefreq)
    else:
        hz = "The freq is {} Hz." .format (thefreq)
        print(hz)
        if np.isnan(thefreq):
             print(average(l))
        else:
            l.append(thefreq)
    # read some more data
    data = wf.readframes(chunk)
    

The freq is 67.56494346774349 Hz.
The freq is 53.47160898562273 Hz.
The freq is 109.61933592491565 Hz.
The freq is 111.57328701005984 Hz.
The freq is 109.81353162518884 Hz.
The freq is 109.63883325931882 Hz.
The freq is 112.1962073787401 Hz.
The freq is 106.40269673904415 Hz.
The freq is 114.04332869025272 Hz.
The freq is 104.9183375955046 Hz.
The freq is 114.23709834095793 Hz.
The freq is 106.03866514171449 Hz.
The freq is 112.27575298404102 Hz.
The freq is 109.95191611962512 Hz.
The freq is 107.41357628655985 Hz.
The freq is 114.3950626572577 Hz.
The freq is 103.66645791463742 Hz.
The freq is 115.95813586985153 Hz.
The freq is 103.32493764507272 Hz.
The freq is 114.66201344495227 Hz.
The freq is 106.46673291115118 Hz.
The freq is 110.47226465558995 Hz.
The freq is 111.638714463267 Hz.
The freq is 106.01332834628454 Hz.
The freq is 114.28611049453058 Hz.
The freq is 104.32698404524223 Hz.
The freq is 114.53551179559652 Hz.
The freq is 105.27925132169958 Hz.
The freq is 113.09354430347

/Users/aki/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
/Users/aki/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan
The freq is nan Hz.
nan


In [5]:
freq_ave=np.average(l)
print(freq_ave)

87.76443353953239


In [6]:
if data:
    stream.write(data)
stream.close()
p.terminate()

In [7]:
def getNearestValue(list, num):
    """
    概要: リストからある値に最も近い値を返却する関数
    @param list: データ配列
    @param num: 対象値
    @return 対象値に最も近い値
    """

    # リスト要素と対象値の差分を計算し最小値のインデックスを取得
    idx = np.abs(np.asarray(list) - num).argmin()
    return list[idx]

In [8]:
json_open = open('bass_freq.json', 'r')
json_load = json.load(json_open)
freq_list =[]
key_list =[]

for v in json_load:
    freq_list.append(v['freq'])
    key_list.append(v['key'])

print(getNearestValue(freq_list, freq_ave))

87.307057858251


In [9]:
freq_num = freq_list.index(getNearestValue(freq_list, freq_ave))

In [10]:
print(key_list[freq_num])

F
